**Генерация новостного текста на английском языке с использованием RNN**


Или пример как не надо создавать фейковые новости

In [ ]:
!pip install requests beautifulsoup4

In [8]:
# Импорт библиотек
import re
import numpy as np
import pickle
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

Для обучения модели я собрала датасет из 500 новостных статей от BBC и The Guardian

In [ ]:
bbc_rss_urls = [
    "https://feeds.bbci.co.uk/news/rss.xml",
    "https://feeds.bbci.co.uk/news/world/rss.xml",
    "https://feeds.bbci.co.uk/news/uk/rss.xml",
    "https://feeds.bbci.co.uk/news/business/rss.xml",
    "https://feeds.bbci.co.uk/news/technology/rss.xml",
    "https://feeds.bbci.co.uk/news/science_and_environment/rss.xml",
    "https://feeds.bbci.co.uk/news/health/rss.xml",
    "https://feeds.bbci.co.uk/news/education/rss.xml",
    "https://feeds.bbci.co.uk/news/entertainment_and_arts/rss.xml",
    "https://feeds.bbci.co.uk/news/politics/rss.xml"
]

def get_bbc_links(rss_url):
    response = requests.get(rss_url)
    soup = BeautifulSoup(response.content, features="xml")
    return [item.link.text for item in soup.find_all("item")]

def get_bbc_article_text(url):
    try:
        r = requests.get(url)
        soup = BeautifulSoup(r.content, 'html.parser')
        article = soup.find('article')
        if article:
            paragraphs = article.find_all('p')
            return ' '.join(p.text for p in paragraphs)
        return None
    except Exception as e:
        print(f"Ошибка {url}: {e}")
        return None

bbc_articles = []
seen_urls = set()

for rss_url in bbc_rss_urls:
    links = get_bbc_links(rss_url)
    for link in links:
        if link in seen_urls or len(bbc_articles) >= 250:
            continue
        seen_urls.add(link)
        text = get_bbc_article_text(link)
        if text and len(text) > 300:
            bbc_articles.append(text)
            print(f"BBC: {len(bbc_articles)} статей собрано.")
        time.sleep(1)

# Сохраняем
with open("bbc_articles.txt", "w", encoding="utf-8") as f:
    for article in bbc_articles:
        f.write(article + "\n" + "="*80 + "\n")

BBC: 1 статей собрано.
BBC: 2 статей собрано.
BBC: 3 статей собрано.
BBC: 4 статей собрано.
BBC: 5 статей собрано.
BBC: 6 статей собрано.
BBC: 7 статей собрано.
BBC: 8 статей собрано.
BBC: 9 статей собрано.
BBC: 10 статей собрано.
BBC: 11 статей собрано.
BBC: 12 статей собрано.
BBC: 13 статей собрано.
BBC: 14 статей собрано.
BBC: 15 статей собрано.
BBC: 16 статей собрано.
BBC: 17 статей собрано.
BBC: 18 статей собрано.
BBC: 19 статей собрано.
BBC: 20 статей собрано.
BBC: 21 статей собрано.
BBC: 22 статей собрано.
BBC: 23 статей собрано.
BBC: 24 статей собрано.
BBC: 25 статей собрано.
BBC: 26 статей собрано.
BBC: 27 статей собрано.
BBC: 28 статей собрано.
BBC: 29 статей собрано.
BBC: 30 статей собрано.
BBC: 31 статей собрано.
BBC: 32 статей собрано.
BBC: 33 статей собрано.
BBC: 34 статей собрано.
BBC: 35 статей собрано.
BBC: 36 статей собрано.
BBC: 37 статей собрано.
BBC: 38 статей собрано.
BBC: 39 статей собрано.
BBC: 40 статей собрано.
BBC: 41 статей собрано.
BBC: 42 статей собрано.
B

In [ ]:
API_KEY = "-----"
guardian_articles = []
page = 1

while len(guardian_articles) < 250:
    url = (
        f"https://content.guardianapis.com/search"
        f"?api-key={API_KEY}&page-size=50&page={page}&show-fields=bodyText"
    )
    r = requests.get(url)
    data = r.json()

    for result in data['response']['results']:
        text = result['fields'].get('bodyText', '')
        if text and len(text) > 300:
            guardian_articles.append(text)
            print(f"Guardian: {len(guardian_articles)} статей собрано.")
            if len(guardian_articles) >= 250:
                break
    page += 1
    time.sleep(1)

# Сохраняем
with open("guardian_articles.txt", "w", encoding="utf-8") as f:
    for article in guardian_articles:
        f.write(article + "\n" + "="*80 + "\n")

Guardian: 1 статей собрано.
Guardian: 2 статей собрано.
Guardian: 3 статей собрано.
Guardian: 4 статей собрано.
Guardian: 5 статей собрано.
Guardian: 6 статей собрано.
Guardian: 7 статей собрано.
Guardian: 8 статей собрано.
Guardian: 9 статей собрано.
Guardian: 10 статей собрано.
Guardian: 11 статей собрано.
Guardian: 12 статей собрано.
Guardian: 13 статей собрано.
Guardian: 14 статей собрано.
Guardian: 15 статей собрано.
Guardian: 16 статей собрано.
Guardian: 17 статей собрано.
Guardian: 18 статей собрано.
Guardian: 19 статей собрано.
Guardian: 20 статей собрано.
Guardian: 21 статей собрано.
Guardian: 22 статей собрано.
Guardian: 23 статей собрано.
Guardian: 24 статей собрано.
Guardian: 25 статей собрано.
Guardian: 26 статей собрано.
Guardian: 27 статей собрано.
Guardian: 28 статей собрано.
Guardian: 29 статей собрано.
Guardian: 30 статей собрано.
Guardian: 31 статей собрано.
Guardian: 32 статей собрано.
Guardian: 33 статей собрано.
Guardian: 34 статей собрано.
Guardian: 35 статей соб

In [9]:
# Загрузка текста и очистка данных
with open("news_corpus.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

# Разбиваем по разделителю строк
raw_articles = raw_text.split("================================================================================") # вот такой вот странный разделитесь у меня в датасете

# Убираем теги [BBC], [Guardian], пустые строки и пробелы
cleaned_articles = []
for article in raw_articles:
    # Удаляем лишние теги
    article = article.replace("[BBC]", "").replace("[Guardian]", "").strip()
    if article:
        cleaned_articles.append(article)

# Проверим результат
print(f"Количество статей: {len(cleaned_articles)}")
print("Пример статьи:", cleaned_articles[0][:500])  # первые 500 символов первой статьи

Количество статей: 500
Пример статьи: This week's G7 summit in Canada will be dominated by war - only not one of those that the world leaders had expected. High on the agenda had been Russia's war against Ukraine and Donald Trump's tariff war against America's trading partners. Instead the three-day gathering in the Rocky Mountains of Alberta will inevitably be focused on war in the Middle East. Israel's decision to attack Iran will force the Group of Seven western powers to spend less time on other issues and instead discuss ways o


In [10]:
# Каждую статью разбиваю на отдельные предложения
sentences = []
for article in cleaned_articles:
    sents = re.split(r'[.!?]', article)
    sents = [s.strip().lower() for s in sents if len(s.strip()) > 10]
    sentences.extend(sents)

In [11]:
# Токенизация и подготовка данных
vocab_size = 4000 # создадим словарь на 4000 наиболее часто встречающихся слов
tokenizer = Tokenizer(num_words=vocab_size, oov_token="<OOV>") # все неизвестные слова заменим на out-of-vocabulary
tokenizer.fit_on_texts(sentences)
# Создание обучающих последовательностей
sequences = tokenizer.texts_to_sequences(sentences)

max_seq_len = 20 # размер входной последовательности
X, y = [], []

for seq in sequences:
    for i in range(1, len(seq)):
        n_gram = seq[max(0, i - max_seq_len):i+1]
        X.append(n_gram[:-1])
        y.append(n_gram[-1])

X = pad_sequences(X, maxlen=max_seq_len) # заполняем недостающие позиции слева
y = np.array(y)  # просто массив целых меток

print(f"Тренировочные сэмплы: {len(X)}")

#  Создание и компиляция модели
model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=64, input_length=max_seq_len))
model.add(LSTM(100)) # из-за ограниченной RAM взяла 100, т.к. на 150 уже не справлялось
model.add(Dense(vocab_size, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.summary()

Тренировочные сэмплы: 521398


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [7]:
model.fit(X, y, batch_size=64, epochs=15, verbose=1)

Epoch 1/15
8147/8147 ━━━━━━━━━━━━━━━━━━━━ 424s 52ms/step - accuracy: 0.1581 - loss: 5.8780
Epoch 2/15
8147/8147 ━━━━━━━━━━━━━━━━━━━━ 421s 49ms/step - accuracy: 0.1914 - loss: 5.1538
Epoch 3/15
8147/8147 ━━━━━━━━━━━━━━━━━━━━ 405s 50ms/step - accuracy: 0.2066 - loss: 4.8649
Epoch 4/15
8147/8147 ━━━━━━━━━━━━━━━━━━━━ 405s 50ms/step - accuracy: 0.2168 - loss: 4.6958
Epoch 5/15
8147/8147 ━━━━━━━━━━━━━━━━━━━━ 436s 49ms/step - accuracy: 0.2244 - loss: 4.5634
Epoch 6/15
8147/8147 ━━━━━━━━━━━━━━━━━━━━ 401s 49ms/step - accuracy: 0.2309 - loss: 4.4592
Epoch 7/15
8147/8147 ━━━━━━━━━━━━━━━━━━━━ 448s 50ms/step - accuracy: 0.2362 - loss: 4.3727
Epoch 8/15
8147/8147 ━━━━━━━━━━━━━━━━━━━━ 405s 50ms/step - accuracy: 0.2418 - loss: 4.2939
Epoch 9/15
8147/8147 ━━━━━━━━━━━━━━━━━━━━ 447s 50ms/step - accuracy: 0.2454 - loss: 4.2339
Epoch 10/15
8147/8147 ━━━━━━━━━━━━━━━━━━━━ 403s 49ms/step - accuracy: 0.2491 - loss: 4.1784
Epoch 11/15
8147/8147 ━━━━━━━━━━━━━━━━━━━━ 401s 49ms/step - accuracy: 0.2530 - loss: 4.12

In [ ]:
# посмотрим на наиболее часто встречающиеся 3-граммы, чтобы сформулировать стартовые фразы
from collections import Counter
from nltk.util import ngrams

# Пример: 3-граммы
three_grams = []
for sentence in sentences:
    tokens = sentence.split()
    three_grams.extend(ngrams(tokens, 3))

common_3grams = Counter(three_grams).most_common(50)
for grams, freq in common_3grams:
    print(" ".join(grams), ":", freq)

one of the : 209
a lot of : 142
as well as : 130
in the uk : 114
the end of : 103
part of the : 95
according to the : 93
some of the : 91
said it was : 87
over: south africa : 84
said in a : 81
told the bbc : 80
in the middle : 79
be able to : 79
to be a : 77
out of the : 76
the middle east : 74
in the first : 72
a couple of : 72
in the us : 70
the trump administration : 70
a number of : 64
the white house : 63
in a statement : 62
there is a : 61
it was a : 61
for the first : 61
the united states : 59
going to be : 59
the first time : 56
this is a : 54
there is no : 54
sign up for : 54
as part of : 53
the number of : 51
it is a : 51
is one of : 50
in the region : 49
the university of : 49
to make a : 48
said he was : 48
at the time : 48
end of the : 46
of the day : 45
a series of : 44
in the world : 44
part of a : 44
at the same : 43
(target 282; markram : 43
there was a : 42


In [ ]:
# Сохранение модели
model.save("rnn_news_generator.keras")
with open("tokenizer.pkl", "wb") as f:
    pickle.dump(tokenizer, f)

# Генерация текста
def generate_text(seed_text, tokenizer, model, maxlen=20, num_words=50, temperature=1.0):
    result = seed_text.lower()
    for _ in range(num_words):
        token_list = tokenizer.texts_to_sequences([result])[0]
        token_list = token_list[-maxlen:]
        token_list = pad_sequences([token_list], maxlen=maxlen)
        prediction = model.predict(token_list, verbose=0)[0]

        predicted_id = sample_with_temperature(prediction, temperature)
        output_word = tokenizer.index_word.get(predicted_id, "")

        if output_word == "" or output_word == "<OOV>":
            continue  # пропускаем <OOV> или неизвестные слова

        result += " " + output_word
    return result

# Составим стартовые фразы
seed_phrases = [
    "the white house", "climate change is", "environmental issues", "the trump administration",
    "the university of oxford researchers", "the prime minister said", "in the world", "the investigation revealed", "social media"
]

for seed in seed_phrases:
    print(f"\n Seed: \"{seed}\"")
    print(generate_text(seed, tokenizer, model, maxlen=30, num_words=50, temperature=0.8)) # с temperature=0.4 получались результаты далекие от смысла


 Seed: "the white house"
the white house is also as the governor began writing that net zero tariffs has become a very different possibility that the us the president is he produced a source of the bill in reducing

 Seed: "climate change is"
climate change is using interest in tax rises added the first scheme would be made for the face of the pandemic the patient and safety mail would sell sewage sludge on its climate crisis to deal with small

 Seed: "environmental issues"
environmental issues in that time lords will also be a source of sustained bills to make it as it was done by the material be taken to be ruled out how to pay a screening health

 Seed: "the trump administration"
the trump administration has become critical of the critical war in gaza as the us president the situation is going to join the new parade to the scale of the zionist regime and us intelligence facilities in the

 Seed: "the university of oxford researchers"
the university of oxford researchers have only 

Результаты генерации текста довольно интересные и в целом соответствуют ожидаемому качеству для RNN, обученной на сравнительно небольшом корпусе. Модель использует клише из новостей — это плюс, например, "a medical care", "the critical war in gaza", "net zero tariffs". Если же есть хорошее удержание контекста в начале предложения, то к концу уже нет. Конечно, это достаточно трудная задача особенно из-за специфичной лексики. Что можно улучшить, но Google Colab ограничивает.